<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/RNN/seq2seq_sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Seq2Seq: Sortowanie
##Import bibliotek

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

TRAINING_SIZE = 50000
NUMBERS_TO_SORT = 10

##Generowanie danych

In [26]:
class CharacterTable:
  
  def __init__(self, chars):
    self.chars = sorted(set(chars))
    self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
    self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

  def encode(self, C, num_rows):
    x = np.zeros((num_rows, len(self.chars)))
    for i, c in enumerate(C):
      x[i, self.char_indices[c]] = 1
    return x

  def decode(self, x, calc_argmax=True):
    if calc_argmax:
      x = x.argmax(axis=-1)
    return ''.join(self.indices_char[x] for x in x)

# Wszystkie liczby, znaki i spacja
chars = '0123456789'
ctable = CharacterTable(chars)

questions = []
expected = []

while len(questions) < TRAINING_SIZE:
  randomize_string = lambda: str(
      ''.join(
          np.random.choice(list('0123456789'))
          for i in range(NUMBERS_TO_SORT)
      )
  )
  query = randomize_string()

  # String to list
  ans = [int(q) for q in query]
  ans = sorted(ans)

  # Sorted list to string
  answer = ''
  for num in ans:
    answer += str(num)
  
  questions.append(query)
  expected.append(answer)

print('Liczba przykładów:', len(questions))
print('Questions: ', questions[:5])
print('Answers: ', expected[:5])

Liczba przykładów: 50000
Questions:  ['8847644026', '3025793945', '1664364979', '2849673969', '3700480495']
Answers:  ['0244466788', '0233455799', '1344666799', '2346678999', '0003445789']


##Wektoryzacja danych

In [27]:
x = np.zeros((len(questions), NUMBERS_TO_SORT, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), NUMBERS_TO_SORT, len(chars)), dtype=np.bool)

for i, sentence in enumerate(questions):
  x[i] = ctable.encode(sentence, NUMBERS_TO_SORT)
for i, sentence in enumerate(expected):
  y[i] = ctable.encode(sentence, NUMBERS_TO_SORT)

indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Dane treningowe:')
print(x_train.shape)
print(y_train.shape)

print('Dane walidacyjne:')
print(x_val.shape)
print(y_val.shape)

Dane treningowe:
(45000, 10, 10)
(45000, 10, 10)
Dane walidacyjne:
(5000, 10, 10)
(5000, 10, 10)


##Budowa modelu

In [38]:
num_layers = 1

model = keras.Sequential()
model.add(layers.LSTM(16, input_shape=(NUMBERS_TO_SORT, len(chars))))
model.add(layers.RepeatVector(NUMBERS_TO_SORT))

for _ in range(num_layers):
  model.add(layers.LSTM(16, return_sequences=True))

model.add(layers.Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 16)                1728      
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 10, 16)            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 10, 16)            2112      
_________________________________________________________________
dense_7 (Dense)              (None, 10, 10)            170       
Total params: 4,010
Trainable params: 4,010
Non-trainable params: 0
_________________________________________________________________


##Trening modelu

In [39]:
epochs = 5
batch_size = 32

for epoch in range(1, epochs + 1):
  print()
  print('Iteracja', epoch)
  model.fit(
      x_train,
      y_train,
      batch_size=batch_size,
      epochs=1,
      validation_data=(x_val, y_val)
  )
  # Wybór 10 losowych próbek ze zbioru walidacyjnego, 
  # abyśmy mogli zobaczyć błędy
  for i in range(10):
    ind = np.random.randint(0, len(x_val))
    rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
    preds = np.argmax(model.predict(rowx), axis=-1)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
      print('☑ ' + guess)
    else:
      print('☒ ' + guess)


Iteracja 1
1407/1407 [==============================] - 17s 10ms/step - loss: 0.8683 - accuracy: 0.7232 - val_loss: 0.3988 - val_accuracy: 0.8993
Q 7543839439 T 3334457899 ☒ 2334557899
Q 2589669405 T 0245566899 ☒ 0145566899
Q 3739211020 T 0011223379 ☑ 0011223379
Q 4230435049 T 0023344459 ☒ 0123344459
Q 8195226339 T 1223356899 ☒ 0223356899
Q 5887675366 T 3556667788 ☒ 1556667788
Q 5740260425 T 0022445567 ☒ 0122445567
Q 9658427089 T 0245678899 ☑ 0245678899
Q 2872732756 T 2223567778 ☒ 2233567777
Q 6575054576 T 0455556677 ☒ 0455566677

Iteracja 2
1407/1407 [==============================] - 13s 9ms/step - loss: 0.2551 - accuracy: 0.9442 - val_loss: 0.1640 - val_accuracy: 0.9680
Q 8464997804 T 0444678899 ☑ 0444678899
Q 2124903773 T 0122334779 ☑ 0122334779
Q 6137035694 T 0133456679 ☑ 0133456679
Q 7719639560 T 0135667799 ☑ 0135667799
Q 1797418294 T 1124477899 ☒ 0124477899
Q 2732399463 T 2233346799 ☑ 2233346799
Q 6638838210 T 0123366888 ☑ 0123366888
Q 3799983347 T 3334778999 ☑ 3334778999
Q 051